In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate

# Load the dataset
file_path = 'combined-dataset/final_reviews_data.csv'
data = pd.read_csv(file_path)

# Encode the 'types' column
label_encoder = LabelEncoder()
data['types_encoded'] = label_encoder.fit_transform(data['types'])

# Tokenize the 'review' column
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['review'])
sequences = tokenizer.texts_to_sequences(data['review'])

# Pad the sequences
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Create the feature set
X = {
    'review': padded_sequences,
    'types': data['types_encoded'].values,
}

# Normalize the sentiment scores
y = data['sentiment'].values


In [13]:
# Define input layers
review_input = Input(shape=(max_sequence_length,), name='review')
types_input = Input(shape=(1,), name='types')

# Define embedding and LSTM layers for review input
review_embedding = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(review_input)
review_lstm = LSTM(128)(review_embedding)

# Define embedding layer for types input
types_embedding = Embedding(input_dim=data['types_encoded'].nunique(), output_dim=10)(types_input)
types_flat = tf.keras.layers.Flatten()(types_embedding)

# Concatenate the review and types embeddings
concatenated = Concatenate()([review_lstm, types_flat])

# Add dense layers for final prediction
dense_1 = Dense(128, activation='relu')(concatenated)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1, activation='linear')(dense_2)

# Create the model
model = Model(inputs=[review_input, types_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit([X['review'], X['types']], y, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 547s 665ms/step - loss: 0.8965 - mae: 0.5329 - val_loss: 0.0898 - val_mae: 0.2247
Epoch 2/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 613s 748ms/step - loss: 0.0527 - mae: 0.1662 - val_loss: 0.0548 - val_mae: 0.1634
Epoch 3/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 646s 789ms/step - loss: 0.0295 - mae: 0.1219 - val_loss: 0.0413 - val_mae: 0.1388
Epoch 4/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 591s 721ms/step - loss: 0.0163 - mae: 0.0901 - val_loss: 0.0374 - val_mae: 0.1339
Epoch 5/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 501s 611ms/step - loss: 0.0120 - mae: 0.0781 - val_loss: 0.0327 - val_mae: 0.1201
Epoch 6/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 505s 617ms/step - loss: 0.0095 - mae: 0.0683 - val_loss: 0.0301 - val_mae: 0.1160
Epoch 7/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 506s 617ms/step - loss: 0.0082 - mae: 0.0647 - val_loss: 0.0278 - val_mae: 0.1123
Epoch 8/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 503s 615ms/step - loss: 0.0064 - mae: 0.0566 - val_loss: 0.0264 - val_mae: 0.1076
Epoch 9/10
819/819 ━━━━━

In [14]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to get recommendations based on a place ID
def get_recommendations(place_id, data, model, top_n=10):
    place_idx = data[data['id'] == place_id].index[0]
    place_review = X['review'][place_idx]
    place_types = X['types'][place_idx]

    # Predict the sentiment for all places
    predicted_sentiments = model.predict([X['review'], X['types']])

    # Calculate similarity
    place_vector = np.concatenate([place_review, [place_types]])
    all_vectors = np.hstack([X['review'], X['types'].reshape(-1, 1)])
    similarities = cosine_similarity([place_vector], all_vectors)[0]

    # Get top N similar places
    similar_indices = np.argsort(similarities)[-top_n:][::-1]
    similar_places = data.iloc[similar_indices]

    return similar_places

# Example usage
place_id = 'ChIJYcGr7GSb0S0RckePBrCWikw'  # Replace with an actual place ID from your dataset
recommendations = get_recommendations(place_id, data, model, top_n=10)
print(recommendations)


1024/1024 ━━━━━━━━━━━━━━━━━━━━ 195s 190ms/step
                                id  \
0      ChIJYcGr7GSb0S0RckePBrCWikw   
26459  ChIJs5sHkDub0S0Rs6_ArAb6GYw   
14966  ChIJoxOsRvY90i0RfdRmmjHBxDs   
2445   ChIJVWVU7lI60i0ROjIjerOky-4   
27601  ChIJ8QkA6MQi0i0RJwJqRqijNpc   
1304   ChIJQTrID1IF0i0REGYdKOXKObk   
3288   ChIJw5Yz3ZU50i0R2RDRqJsUvAs   
26714  ChIJ5ZNHoDeJ0S0RhyP0FmUFXBQ   
7369   ChIJ-UX-z8wg0i0R2t1WpZDLVDA   
14808  ChIJ0__EiSM90i0Rh-u4ELIOCcQ   

                                                types review_number  \
0                                      hotel, lodging      review 1   
26459                                restaurant, food      review 5   
14966            italian_restaurant, restaurant, food      review 3   
2445          indonesian_restaurant, restaurant, food      review 1   
27601  indonesian_restaurant, restaurant, food, store      review 5   
1304       restaurant, coffee_shop, cafe, food, store      review 1   
3288                                 

In [18]:
# Save the model
model.save('39_test_model.keras')

In [19]:
# Load the model
model = tf.keras.models.load_model('39_test_model.keras')

C:\Users\anton\PycharmProjects\BangkitStuff\capstone-ngelana\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 13 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [38]:
df_review = pd.read_csv('combined-dataset/final_reviews_data.csv')
df_place = pd.read_csv('combined-dataset/combined_datasetV2.csv')

random_place = df_review.sample(1)
rand_id = random_place['id'].values[0]
print(f'Random place :{rand_id}', df_place[df_place['id'] == rand_id]['name'].values[0])

recommendations = get_recommendations(rand_id, df_review, model, top_n=10)

Random place :ChIJlyzLUMdG0i0R9uZM0KyN8WA Four Points by Sheraton Bali, Kuta
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 254s 248ms/step


In [39]:

# Merge the recommendations with place names based on 'id'
merged_recommendations = recommendations.merge(df_place, on='id')
# sort reccomendations by sentiment
sorted_reccomendations = merged_recommendations.sort_values(by='sentiment', ascending=False)
# Print the recommendations with place names with out rand_ind
print(sorted_reccomendations[['name','types_x', 'rating']])

                                 name  \
1                        Villa Anjing   
8     KŌWHAI Restaurant - Nusa Penida   
9                    HOME CAFE CANGGU   
7                 Paca Bali Transport   
0  Four Points by Sheraton Bali, Kuta   
6                      Warung Pananda   
2                       What The Crab   
5                     Bali Dream Ubud   
4               Babi Guling Men Janji   
3                   Kajeng Rice Field   

                                   types_x  rating  
1                     guest_house, lodging     3.8  
8   restaurant, breakfast_restaurant, food     4.7  
9           coffee_shop, cafe, food, store     4.3  
7                               car_rental     5.0  
0                           hotel, lodging     4.5  
6                         restaurant, food     4.7  
2     seafood_restaurant, restaurant, food     4.6  
5             resort_hotel, hotel, lodging     4.0  
4  indonesian_restaurant, restaurant, food     4.4  
3                 